In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import pint
import os
import pandas as pd
import itertools

In [ ]:
unit = pint.UnitRegistry()

RAW Data

In [ ]:

file_path1 = "DATA/Magnetic_Field_Inside_Solenoid_Table_3_12.csv"
file_path2 = "DATA/Magnetic_Field_Outside_Solenoid_Table_3_1.csv"

fig_path = "output_plots/"

if not os.path.exists(fig_path):
    os.makedirs(fig_path)

inside_feild_data = pd.read_csv(file_path1)
outside_feild_data = pd.read_csv(file_path2)



# Display the data
print("(Table 3.12):")
print(inside_feild_data.head())

print("\n(Table 3.1):")

outside_feild_data["tan_alpha"] = np.tan(np.radians(outside_feild_data["alpha"]))
B_y = 26 * unit.microtesla
B = lambda alpha: B_y * (np.tan(np.radians(alpha)) *unit.dimensionless)
outside_feild_data["B_muT"] = B(outside_feild_data["alpha"])

print(outside_feild_data.head())


In [ ]:
inside_feild_data.to_csv(file_path1, index=False)  # Update the file (Table 3.12)
outside_feild_data.to_csv(file_path2, index=False)  # Update the file (Table 3.1)

print(outside_feild_data["B_muT"][2])

In [ ]:
outside_ex_current = outside_feild_data["I_A"].to_numpy()
outside_ex_tangante_alpha = outside_feild_data["tan_alpha"].to_numpy()

model_outside = LinearRegression()
model_outside.fit(outside_ex_tangante_alpha.reshape(-1,1), outside_ex_current)
outside_line_space = np.linspace(min(outside_ex_tangante_alpha), max(outside_ex_tangante_alpha))
reg = model_outside.predict(outside_line_space.reshape(-1,1))

plt.figure(dpi=300)

plt.grid(True)
plt.scatter(outside_ex_tangante_alpha, outside_ex_current,c ="b", label="Data")
plt.plot(outside_line_space, reg, c="r", label="Model")

plt.ylabel("$I$ $(A)$")
plt.xlabel("$tan\\alpha$")
plt.title("$I = f(\\alpha)$")
plt.tight_layout()
plt.legend()

plt.savefig(fig_path+"current_vs_tan_alpha.jpg",dpi=300)
plt.show()

In [ ]:
for i in inside_feild_data.columns:
    print(i)

In [ ]:
for column_name in inside_feild_data.columns:
    if column_name != "x (mm)":
        print(i)

In [ ]:
Degree = 6
X_mm = inside_feild_data["x (mm)"].to_numpy().reshape(-1,1)
poly_features = PolynomialFeatures(degree=Degree)
X_poly = poly_features.fit_transform(X_mm)
colors = itertools.cycle(['b', 'g', 'r', 'c', 'm', 'y', 'k'])

ALL_mid_pints = []

for column_name in inside_feild_data.columns:
    if column_name != "x (mm)":
        plt.figure(dpi=300)
        color = next(colors)

        y = inside_feild_data[column_name]
        model = LinearRegression()
        model.fit(X_poly, y)
        line_space = np.linspace(min(X_mm), max(X_mm),1000).reshape(-1,1)
        poly_line_space = poly_features.fit_transform(line_space)
        prediction = model.predict(poly_line_space)

        mid_x = poly_features.fit_transform(np.array([110]).reshape(-1,1))
        mid_point_B = (model.predict(mid_x))
        
        plt.scatter(X_mm, y, label=column_name.replace(" B (mT)", "").replace("turns ", ""), c=color)
        plt.plot(line_space,prediction, label=column_name.replace(" B (mT)", "").replace("turns ", "")+" Model", c=color)
        
        plt.grid(True)
        plt.ylabel(r"$B (mT)$")
        plt.xlabel(r"$x$ $(mm)$")
        plt.legend()        
        file_name= column_name.replace("turns ", "").replace(":", "_").replace(" ", "_")
        plt.savefig(fig_path+f"{file_name}.jpg", dpi=300)
        plt.show()
        print(f"Mid_point = {mid_point_B}")
        ALL_mid_pints.append((column_name.strip(), mid_point_B))


In [ ]:
len(ALL_mid_pints) #5 means for each configuration

In [ ]:
ALL_mid_pints

In [ ]:
ALL_mid_pints_n_integered = [(int(n[0][2:5]), n[1] ) for n in ALL_mid_pints]

In [ ]:
ALL_mid_pints_n_integered

In [ ]:
x, y = [], []
for i in ALL_mid_pints_n_integered:
    x.append(i[0]);y.append(i[1])


In [ ]:
x, y = np.array(x), np.array(y).reshape(1,-1)[0]*10**-3 # turns, tesla
print(x," -|- \n",  y)

In [ ]:
model_mu = LinearRegression()
model_mu.fit(x.reshape(-1,1),y)
line_space_mu = np.linspace(min(x), max(x), 1000).reshape(-1,1)
predictions_mu = model_mu.predict(line_space_mu)


plt.figure(dpi=300)
plt.scatter(x,y, c = "b", label="Data")
plt.plot(line_space_mu,predictions_mu, c="r", label="Model")
plt.grid(True)
plt.legend()
plt.xlabel("N (turns)")
plt.ylabel("B ($T$)")
plt.title("B(N) - for a fixed x = 110 cm")
plt.savefig("output_plots/B_vs_n_110cm.png")

slope = model_mu.coef_[0]  # Coefficient of the fitted model
print(f"The slope of the model is {slope}")

In [ ]:
# B = \mu * n * I => \mu = B / (n * I) 
I = 0.5 * unit.ampere
l = 160 * unit.millimeter
mu = lambda B, N: B*l/(N*I)
i = 0
p = ALL_mid_pints_n_integered[i][1][0] * unit.millitesla
N = ALL_mid_pints_n_integered[i][0]
mu(p, N).to(unit.henry/unit.meter)

In [ ]:
mu_values = []
for N, B in ALL_mid_pints_n_integered:
    mu_values.append(mu(B * unit.millitesla, N).to(unit.henry/unit.meter))

In [ ]:
mu_values

In [ ]:

Degree = 6
X_mm = inside_feild_data["x (mm)"].to_numpy().reshape(-1, 1)
poly_features = PolynomialFeatures(degree=Degree)
X_poly = poly_features.fit_transform(X_mm)

# Define marker and line styles
markers = itertools.cycle(['o', '^', '+',  '*','x','s', 'D', 'v', '<', '>', 'P',  ',', '1', '2', '3'])
linestyles = itertools.cycle(['-', '--', ':', '-.', '-', '--', ':', '-.', '-'])

# Create a single figure before the loop
plt.figure(figsize=(13,9),dpi=300)

for column_name in inside_feild_data.columns:
    if column_name != "x (mm)":
        # Cycle through markers and line styles
        marker = next(markers)
        linestyle = next(linestyles)

        y = inside_feild_data[column_name]
        model = LinearRegression()
        model.fit(X_poly, y)
        
        # Generate a space for plotting the regression line
        line_space = np.linspace(min(X_mm), max(X_mm), 1000).reshape(-1, 1)
        poly_line_space = poly_features.transform(line_space)
        prediction = model.predict(poly_line_space)

        # Plot the scatter points with a specific marker
        plt.scatter(X_mm, y, label=column_name.replace(" B (mT)", "").replace("turns ", ""), marker=marker)

        # Plot the regression line with a specific line style
        plt.plot(line_space, prediction, label=f'{column_name.replace(" B (mT)", "").replace("turns ", "")} Model', linestyle=linestyle)

# Customize the plot after the loop
plt.grid(True)
plt.ylabel(r"$B (mT)$")
plt.xlabel(r"$x$ $(mm)$")
plt.legend()        
plt.savefig(fig_path+"Magnatic_Feilds_vs_distance.jpg",dpi=300)
plt.show()


In [25]:
results = {
    "outside_field_data": outside_feild_data.head(),
    "inside_field_data_columns": list(inside_feild_data.columns),
    "midpoints": ALL_mid_pints_n_integered,
    "mu_values": mu_values,
    "slope_BN": slope,
}

# Display the results
results

{'outside_field_data':    I_A  alpha  tan_alpha      B_muT
 0  0.0      0   0.000000   0.000000
 1  0.2     14   0.249328   6.482528
 2  0.4     26   0.487733  12.681047
 3  0.6     36   0.726543  18.890106
 4  0.8     48   1.110613  28.875925,
 'inside_field_data_columns': ['x (mm)',
  'n:75 turns 2a:26mm B (mT) ',
  'n:150 turns 2a:26mm B (mT)',
  'n:300 turns 2a:26mm B (mT)',
  'n:300 turns 2a:33mm B (mT)',
  'n:300 turns 2a:41mm  B (mT)'],
 'midpoints': [(75, array([0.28183772])),
  (150, array([0.53421586])),
  (300, array([1.09196126])),
  (300, array([1.09208184])),
  (300, array([1.03782164]))],
 'mu_values': [<Quantity([1.20250759e-06], 'henry / meter')>,
  <Quantity([1.1396605e-06], 'henry / meter')>,
  <Quantity([1.16475868e-06], 'henry / meter')>,
  <Quantity([1.1648873e-06], 'henry / meter')>,
  <Quantity([1.10700975e-06], 'henry / meter')>],
 'slope_BN': np.float64(3.5399557507249617e-06)}

In [30]:
((slope*unit.tesla)*l/I).to(unit.henry/unit.meter)

<Quantity(1.13278584e-06, 'henry / meter')>